In [ ]:
%%sparql --store-to results
# Query 3: Get predicate object pairs based on subject list from Query 2
SELECT ?s ?p ?o   
WHERE {
    {
        # Query 2: Filter subject list from Query 1 to keep subjects with more than 1 statement
        SELECT ?s 
        WHERE {
            {
                # Query 1: Count statements per subject
                SELECT ?s (count(*) AS ?subject_statements) 
                WHERE {?s ?p ?o} 
                GROUP BY ?s
            }
            FILTER (?subject_statements > 1)
        }
    }
    {?s ?p ?o} 
}
ORDER BY ?s

In [ ]:
# Flatten results into array of predicate object pairs per subjct
import urllib.parse

triples = results['results']['bindings']
current_subject = None
matrix_of_pred_obj_pairs_all_subj = []
array_of_pred_obj_pairs_per_subj = []
for triple in triples:
    if current_subject == None or current_subject != triple['s']['value']:
        print(triple['s']['value'])
        # Push this subjects pred obj pairs to the matrix
        if len(array_of_pred_obj_pairs_per_subj) > 0:
            print(array_of_pred_obj_pairs_per_subj)
            matrix_of_pred_obj_pairs_all_subj.append(array_of_pred_obj_pairs_per_subj)
        
        # Set new subject and clear subj pairs array
        current_subject = triple['s']['value']
        array_of_pred_obj_pairs_per_subj = []
    
    # Add pred obj pairs to array
    predicate = urllib.parse.unquote_plus(triple['p']['value']).replace('urn:', '')
    object = urllib.parse.unquote_plus(triple['o']['value']).replace('urn:', '')
    array_of_pred_obj_pairs_per_subj.append(predicate + ' ' + object)   

In [ ]:
# Run Frequent Pattern Growth (FP-Growth) Algorithm
# This finds pairs of statements that are found to be true accross different subjects

# For a simple example of fpgrowth alg, check out https://www.kaggle.com/code/rjmanoj/fp-growth-algorithm-frequent-itemset-pattern

import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth
from IPython.display import display, HTML

te = TransactionEncoder()
te_ary = te.fit(matrix_of_pred_obj_pairs_all_subj).transform(matrix_of_pred_obj_pairs_all_subj)
df = pd.DataFrame(te_ary, columns=te.columns_)

fpgrowth_df = fpgrowth(df, min_support=0.002, use_colnames=True)

pd.set_option("display.max_colwidth", 10000)
fpgrowth_df[fpgrowth_df.itemsets.str.len() > 1]